<a href="https://colab.research.google.com/github/mskbasha/Sentiment_analyis/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
import torch 
import math
import pandas as pd 
import numpy as np
from torch.nn import functional as F

# Reading Data

In [2]:
from zipfile import ZipFile
with ZipFile("/content/drive/MyDrive/Chrome OS/Coding data/archive.zip",'r') as f:
    f.extractall()

In [4]:
data = pd.read_csv("/content/train.csv",encoding='ISO-8859-1')

In [5]:
data.head(4)

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164


In [ ]:
class MultiHeadedAttention(torch.nn.Module):
    def __init__(self,heads,seq_length,dim):
        super().__init__()
        assert dim % heads==0
        self.heads = heads
        self.dim = dim
        self.qkv_dim = heads//dim
        self.ff = torch.nn.Linear(self.dim,self.dim,bias = False)
        self.MHA = torch.nn.MultiheadAttention(dim,heads,batch_first=True)
        self.o_proj = torch.nn.Linear(dim,dim,bias = False)
        self.layernorm = torch.nn.LayerNorm([seq_length,dim])
    def __call__(self,x,pad_mask=None,future_mask=None):
        self.q = self.ff(x[:,:1,:])
        self.k = self.ff(x)
        self.v = self.ff(x)
        output,_ = self.MHA(self.q,self.k,self.v,key_padding_mask=pad_mask,attn_mask=future_mask)
        return self.layernorm(output)
class Encoder(torch.nn.Module):
    def __init__(self,no_blocks,heads,seq_length,dim):
        super().__init__()
        self.no_blocks = no_blocks
        self.MHA = MultiHeadedAttention(heads,seq_length,dim)
        self.ff = torch.nn.Linear(dim,dim)
    def __call__(self,x,pad_mask = None):
        
        for i in range(self.no_blocks):
            output = self.MHA(x,pad_mask=pad_mask)
            output += self.ff(x+output)
        return output
class Decoder(torch.nn.Module):
    def __init__(self,no_blocks,heads,seq_length,dim,future_mask=None):
        super().__init__()
        self.seq_length = seq_length
        self.no_blocks = no_blocks
        self.MHA = MultiHeadedAttention(heads,seq_length,dim)
        self.ff = torch.nn.Linear(dim,dim)
    def _create_mask(self,iteration): 
        mask = torch.ones(self.seq_length, self.seq_length)
        mask = mask.triu_(diagonal=1 + iteration)
        return mask
    def __call__(self,x,future_mask=None):
        future_mask = self._create_mask(1)
        for i in range(self.no_blocks):
            output = self.MHA(x,future_mask=future_mask)
            output += self.ff(x+output)
        return output
# class Transformer()
x = Decoder( 1,2,5,6)(torch.normal(0,1,size = (2,5,6)),torch.normal(0,1,size = (1,5)))
x.shape

RuntimeError: ignored

tensor([[False, False, False, False, False],
        [False, False, False, False, False],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True]])

In [ ]:
masked_qkt = torch.tensor([[1,1,0],[2,1,0],[1,3,0]])
values = torch.tensor([[1,2,3,4],[4,5,6,7],[8,9,2,1]])
torch.matmul(masked_qkt,values)

tensor([[ 5,  7,  9, 11],
        [ 6,  9, 12, 15],
        [13, 17, 21, 25]])

NameError: ignored

In [ ]:
torch.sqrt(torch.tensor([1,2,3,4,5])).masked_fill(torch.tensor(  [True,True,False,False,False])   , float('-inf'))

tensor([  -inf,   -inf, 1.7321, 2.0000, 2.2361])

In [ ]:
k = torch.tensor([[
    [1,2,0],
    [3,0,5],
    [6,0,1],
    [0,1,2],
],
[
    [1,2,0],
    [3,0,5],
    [6,0,1],
    [0,1,2],
]])
torch.cat([k[0,:,:],k[1,:,:]],dim=1)

tensor([[1, 2, 0, 1, 2, 0],
        [3, 0, 5, 3, 0, 5],
        [6, 0, 1, 6, 0, 1],
        [0, 1, 2, 0, 1, 2]])

In [ ]:
torch.normal(0,1,size=(4,3,3,3)).masked_fill(k[:,None,None,:]==0,float('-inf')).shape

torch.Size([4, 3, 3, 3])

In [ ]:
torch.matmul(torch.normal(0,1,size=(10,5,10,10)),torch.normal(0,1,size = (10,5,10,5))).shape

torch.Size([10, 5, 10, 5])

In [ ]:
torch.tensor([[1,2,3],[3,4,5]])

tensor([[1, 2, 3],
        [3, 4, 5]])

In [ ]:
torch.tensor([[1,2,3],[3,4,5]]).reshape(3,2)

tensor([[1, 2],
        [3, 3],
        [4, 5]])

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention,LayerNormalization,Dense
import tensorflow as tf

def encoder_block(input_tensor,emb_dim,num_heads):
    q, k, v= Dense(emb_dim)(input_tensor), Dense(emb_dim)(input_tensor), Dense(emb_dim)(input_tensor)
    output = MultiHeadAttention(num_heads=num_heads, key_dim=emb_dim)(q,k,v)
    output = LayerNormalization()(output)
    res = input_tensor+output
    output = Dense(emb_dim)(res)
    output = output+res
    return q,k,v,output
input_tensor = tf.keras.layers.Input((5,12))
output = input_tensor
for i in range(10): 
    q,k,v, output = encoder_block(output,12,2)

model = keras.Model(inputs=input_tensor, outputs=output)

In [ ]:
k = tf.random.normal((10,5,12),0,1)
model.predict(k).shapex

1/1 [==============================] - 2s 2s/step


(10, 5, 12)

In [ ]:
import keras
keras.layers.Dense(1)(tf.random.normal((10,10,),0,1)).shape

TensorShape([10, 1])